In [ ]:
import pickle
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

import keras
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Dropout, PReLU

In [ ]:
BINARY_DATA = (
    'iris_binary_training.csv'
)
REGRESSION_DATA = (
    'boston_housing.csv'
)

bin_X = pd.read_csv(BINARY_DATA)
bin_y = bin_X.pop('Species')

reg_X = pd.read_csv(REGRESSION_DATA)
reg_y = reg_X.pop('MEDV')

In [ ]:
bin_model = Sequential([
    Dense(50, input_dim=bin_X.shape[1]),
    Activation('relu'),
    Dropout(0.2),
    Dense(100),
    PReLU(),
    Dense(1),
    Activation('sigmoid')
])
bin_model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])
target_encoder = LabelEncoder()
target_encoder.fit(bin_y)
bin_model.fit(bin_X, target_encoder.transform(bin_y), epochs=10, batch_size=32)

reg_model = Sequential([
    Dense(50, input_dim=reg_X.shape[1]),
    Activation('relu'),
    Dropout(0.2),
    Dense(100),
    PReLU(),
    Dense(1),
])
reg_model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['accuracy'])
reg_model.fit(reg_X, reg_y, epochs=10, batch_size=32)

In [ ]:
print(bin_model.predict(pd.DataFrame(bin_X)))
print(reg_model.predict(pd.DataFrame(reg_X)))

In [ ]:
bin_model.save('keras_bin.h5')
reg_model.save('keras_reg.h5')